In [1]:
from jupyter_core.application import JupyterApp
from traitlets import Unicode
from traitlets.config import Configurable, Config

# Creating a Configurable class

In [2]:
class A(Configurable):
    my_trait = Unicode("default").tag(config=True)
    
    def _repr_markdown_(self):
        return f"**{self.__class__.__name__}**.*my_trait*: {self.my_trait}"

In [3]:
config_dict = {
    "A":{
        "my_trait": "Not the default"    
    }
}

config = Config(config_dict)
A(config=config)

**A**.*my_trait*: Not the default

In [4]:
A(my_trait="I set this in my `__init__` through \*\*kwargs.")

**A**.*my_trait*: I set this in my `__init__` through \*\*kwargs.

# Subclassing Configurable classes

In [5]:
class B(A):
    pass

display(B())

**B**.*my_trait*: default

## Subclasses inherit parent's settings

In [6]:
display(B(config=config))

**B**.*my_trait*: Not the default

## Set traits with  "." access on Config objects

In [7]:
config.B.my_trait="B now has a new value"
display(type(config), config)

traitlets.config.loader.Config

{'A': {'my_trait': 'Not the default'},
 'B': {'my_trait': 'B now has a new value'}}

## Specificity wins

In [8]:
B(config=config)

**B**.*my_trait*: B now has a new value

## Configuration for classes that can't be Configurable

E.g., `notebook.base.handlers.APIHandler` will break if you multiply inherit `Configurable`

Solution:

- Create a Configurable class next to the nonconfigurable class
- Pass config object into nonconfigurable class
- In the nonconfigurable class, instantiate the Configurable class with the config object

In [ ]:
from notebook.base.handlers import APIHandler

class HandlerSidecar(Configurable):
    auth_token = Unicode("auth")
    
class MyHandler(APIHandler):
    def custom_endpoint(self):
        # self.config is a config object from NotebookWebApp
        sidecar = HandlerSidecar(config=self.config)
        self.auth_token = sidecar.auth_token


## And now back to the slides!
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


# Finding config files

## `jupyter --paths` shows the directories 

In [9]:
!jupyter --paths

config:
    /Users/mpacer/.jupyter
    /Users/mpacer/miniconda3/envs/dev/etc/jupyter
    /usr/local/etc/jupyter
    /etc/jupyter
data:
    /Users/mpacer/Library/Jupyter
    /Users/mpacer/miniconda3/envs/dev/share/jupyter
    /usr/local/share/jupyter
    /usr/share/jupyter
runtime:
    /Users/mpacer/Library/Jupyter/runtime


## JupyterApp defines the config search targets

### `App.name` defines the config file name

**NB**:
`-` is replaced with `_`.

In [10]:
class SimpleConfig(JupyterApp):
    name="jupyter-simple"
    
    @classmethod
    def show_config_path(cls):
        test = cls()
        for path in test.config_file_paths:
            print(f"{path}/{test.config_file_name}.py")
            print(f"{path}/{test.config_file_name}.json")
            print()


In [11]:
test = SimpleConfig.show_config_path()

/Users/mpacer/jupyter/config_talk/jupyter_simple_config.py
/Users/mpacer/jupyter/config_talk/jupyter_simple_config.json

/Users/mpacer/.jupyter/jupyter_simple_config.py
/Users/mpacer/.jupyter/jupyter_simple_config.json

/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_simple_config.py
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_simple_config.json

/usr/local/etc/jupyter/jupyter_simple_config.py
/usr/local/etc/jupyter/jupyter_simple_config.json

/etc/jupyter/jupyter_simple_config.py
/etc/jupyter/jupyter_simple_config.json



## Examples of this convention in the wild

e.g.,:
- for `jupyter notebook`:  
  `jupyter_notebook_config.py`

- for `jupyter nbconvert`:  
  `jupyter_nbconvert_config.json`

# `jupyter config`:  A new app!

Jupyter config helps you solve lots of config file issues

```
pip install jupyter_config
```

## Find all config files with `jupyter config list`

In [12]:
!jupyter config list

jupyter_test_config.py
jupyter_conflict_config.py
/Users/mpacer/.jupyter/jupyter_notebook_config.json
/Users/mpacer/.jupyter/jupyter_console_config.py
/Users/mpacer/.jupyter/jupyter_notebook_config.py
/Users/mpacer/.jupyter/jupyter_nbconvert_config.py
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_notebook_config.json
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_notebook_config.py
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_conflict_config.py
/Users/mpacer/.jupyter/nbconfig/edit.json
/Users/mpacer/.jupyter/nbconfig/notebook.json
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/nbconfig/edit.json
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/nbconfig/notebook.json
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/nbconfig/terminal.json
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/nbconfig/tree.json
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/nbconfig/notebook.d/widgetsnbextension.json


# Loading config into an app

In [13]:
class TestConfig(SimpleConfig):
    name="jupyter_test"

    def start(self):
        self.a = A(parent=self)
        self.b = B(parent=self)
        
    def get_started(self):
        # self.initialize loads the config
        # self.start is where you apply app specific logic
        self.initialize([])
        self.start()
        return self

    def _ipython_display_(self):
        display(self.a)
        display(self.b)


In [14]:
TestConfig.show_config_path()

/Users/mpacer/jupyter/config_talk/jupyter_test_config.py
/Users/mpacer/jupyter/config_talk/jupyter_test_config.json

/Users/mpacer/.jupyter/jupyter_test_config.py
/Users/mpacer/.jupyter/jupyter_test_config.json

/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_test_config.py
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_test_config.json

/usr/local/etc/jupyter/jupyter_test_config.py
/usr/local/etc/jupyter/jupyter_test_config.json

/etc/jupyter/jupyter_test_config.py
/etc/jupyter/jupyter_test_config.json



In [15]:
TestConfig().get_started()

**A**.*my_trait*: In /Users/mpacer/jupyter/config_talk/jupyter_test_config.py, I set A.

**B**.*my_trait*: In /Users/mpacer/jupyter/config_talk/jupyter_test_config.py, I set B.

In [16]:
%pycat /Users/mpacer/jupyter/config_talk/jupyter_test_config.py

c.A.my_trait = f"In {__file__}, I set A."
c.B.my_trait = f"In {__file__}, I set B."


# Specificity wins again!

In [17]:
class ConflictConfig(TestConfig):
    name = "jupyter_conflict"

ConflictConfig.show_config_path()

/Users/mpacer/jupyter/config_talk/jupyter_conflict_config.py
/Users/mpacer/jupyter/config_talk/jupyter_conflict_config.json

/Users/mpacer/.jupyter/jupyter_conflict_config.py
/Users/mpacer/.jupyter/jupyter_conflict_config.json

/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_conflict_config.py
/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_conflict_config.json

/usr/local/etc/jupyter/jupyter_conflict_config.py
/usr/local/etc/jupyter/jupyter_conflict_config.json

/etc/jupyter/jupyter_conflict_config.py
/etc/jupyter/jupyter_conflict_config.json



In [18]:
conflict = ConflictConfig().get_started()
display(conflict)

**A**.*my_trait*: In /Users/mpacer/jupyter/config_talk/jupyter_conflict_config.py, I set A.

**B**.*my_trait*: In /Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_conflict_config.py location, I set B.

## Find config settings with `jupyter config search`

In [24]:
!jupyter config search my_trait

jupyter_test_config.py
1: c.A.my_trait = f"In {__file__}, I set A."
2: c.B.my_trait = f"In {__file__}, I set B." 

jupyter_conflict_config.py
1: c.A.my_trait = f"In {__file__}, I set A." 

/Users/mpacer/miniconda3/envs/dev/etc/jupyter/jupyter_conflict_config.py
1: c.B.my_trait = f"In {__file__} location, I set B." 

/usr/local/etc/jupyter/jupyter_conflict_config.py
1: c.A.my_trait = f"I am not loaded because I'm overwritten locally." 



# Setting traits from the command line

Trait values are also surfaced from the command line:

```
jupyter notebook --NotebookApp.port=8889 --NotebookApp.open_browser=False
```

Shortened versions are sometimes available via `aliases` and `flags`:

```
jupyter notebook --port=8889 --no-browser
```

## And now back to the slides!
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>
